# Class Timetable usando CSP

---

## Projeto de Fundamentos de Inteligência Artificial

---

## 1. Introdução

### 1.1 Equipa do Projeto

**Autores:**
- Alexandre Santos (Nº 33992)
- Jorge Cunha (Nº 30786)

**Instituição:** Instituto Politécnico do Cávado e do Ave (IPCA)  
**Mestrado:** Inteligência Artificial Aplicada  
**Unidade Curricular:** Fundamentos de Inteligência Artificial  
**Data:** Outubro 2025

### 1.2 Contexto

O escalonamento de horários académicos é um problema recorrente nas instituições de ensino superior. A cada ano letivo, as equipas administrativas do IPCA enfrentam dificuldades significativas na criação de horários de aulas que satisfaçam todas as restrições relacionadas com professores, turmas, salas e recursos.

Este é um problema clássico de **otimização combinatória** que pode ser formulado como um **Constraint Satisfaction Problem (CSP)**, onde é necessário encontrar uma atribuição de aulas a slots temporais que satisfaça um conjunto de restrições obrigatórias e preferenciais.

### 1.3 Objetivo do Projeto

Este projeto visa desenvolver uma ferramenta baseada em **Constraint Satisfaction Problem (CSP)** para automatizar o processo de geração de horários de aulas para cursos de licenciatura, respeitando:

- **Hard Constraints** (restrições obrigatórias): disponibilidade de professores, conflitos de turma, duração das aulas, limitações de salas, etc.
- **Soft Constraints** (restrições preferenciais): minimização de dias de aula, consecutividade de aulas, otimização de salas, etc.

**Objetivos específicos:**
1. Modelar o problema de escalonamento como CSP
2. Implementar as restrições hard e soft definidas
3. Encontrar todas as soluções válidas
4. Selecionar a melhor solução através de função de avaliação
5. Apresentar resultados de forma clara e organizada

### 1.4 Ferramentas Utilizadas

- **Python 3**: Linguagem de programação
- **python-constraint**: Biblioteca especializada em resolução de CSP
- **Jupyter Notebook**: Ambiente interativo para desenvolvimento e apresentação

### 1.5 Estrutura do Notebook

Este notebook está organizado nas seguintes secções:

1. **Introdução** - Contexto, objetivos e equipa
2. **Formulação do Problema como CSP** - Variáveis, domínios, restrições e heurísticas
3. **Implementação do Agente** - Código Python com comentários detalhados
4. **Execução e Resultados** - Resolução do CSP, função de avaliação e visualização
5. **Análise Crítica** - Avaliação da qualidade, performance e melhorias futuras
6. **Conclusão** - Síntese dos resultados e aprendizagens

---

## 2. Formulação do Problema como CSP

### 2.1 Definição de CSP

Um **Constraint Satisfaction Problem (CSP)** é definido por:

- **Variáveis**: Conjunto de variáveis de decisão X = {X₁, X₂, ..., Xₙ}
- **Domínios**: Conjunto de valores possíveis para cada variável D = {D₁, D₂, ..., Dₙ}
- **Restrições**: Conjunto de restrições C que limitam as combinações de valores

### 2.2 Variáveis do Problema

Cada variável representa uma **aula** de uma **unidade curricular (UC)** numa **turma**:

- Formato: `UC_turma_aula1` e `UC_turma_aula2`
- Exemplo: `UC11_t01_aula1`, `UC11_t01_aula2`
- Total: 3 turmas × 5 UCs × 2 aulas = **30 variáveis**

### 2.3 Domínios

Cada variável pode ser atribuída a qualquer **slot temporal**:

- Domínio: {1, 2, 3, ..., 20}
- 20 slots = 5 dias × 4 blocos de 2 horas
- Mapeamento:
  - Slots 1-4: Segunda-feira
  - Slots 5-8: Terça-feira
  - Slots 9-12: Quarta-feira
  - Slots 13-16: Quinta-feira
  - Slots 17-20: Sexta-feira

### 2.4 Restrições (Constraints)

#### Hard Constraints (Obrigatórias)

1. **Duração**: Todas as aulas duram 2 horas
2. **Frequência**: Cada UC tem 2 aulas por semana
3. **Limite diário**: Máximo 3 aulas por dia por turma
4. **Disponibilidade**: Professores só dão aulas nos slots disponíveis
5. **Conflito de professor**: Um professor não pode dar aulas simultâneas
6. **Conflito de turma**: Uma turma não pode ter aulas simultâneas
7. **Salas específicas**: Algumas UCs requerem salas específicas (Lab01)
8. **Aulas online**: Aulas online (máximo 3) devem estar no mesmo dia

#### Soft Constraints (Preferenciais)

1. **Dias distintos**: As 2 aulas da mesma UC devem estar em dias diferentes
2. **Quatro dias**: Preferir horários com apenas 4 dias de aulas por turma
3. **Consecutividade**: Aulas consecutivas no mesmo dia (sem "buracos")
4. **Minimizar salas**: Reduzir o número de salas diferentes por turma

### 2.5 Heurísticas Aplicadas

O solver `python-constraint` utiliza:

- **Backtracking**: Algoritmo de busca com retrocesso
- **Constraint Propagation**: Redução do domínio baseada em restrições
- **AllDifferentConstraint**: Otimização para restrições de diferença
- **Função de Avaliação**: Seleção da melhor solução entre múltiplas soluções válidas

---

## 3. Implementação do Agente

### 3.1 Importação de Bibliotecas

In [1]:
# Importação das bibliotecas necessárias
from constraint import *
import datetime
import time

print("Bibliotecas importadas com sucesso!")
print(f"Data e Hora: {datetime.datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}")

Bibliotecas importadas com sucesso!
Data e Hora: 27/10/2025 às 18:09:02


### 3.2 Definição dos Dados do Problema

Todos os dados são definidos diretamente no código (sem ficheiros externos).

In [2]:
# ===== DEFINIÇÃO DOS DADOS DO PROBLEMA =====

# Dataset baseado no enunciado fornecido
# Todos os dados são definidos diretamente no código (sem ficheiros externos)

# Estas estruturas de dados representam o problema real de escalonamento
# que queremos resolver. Cada estrutura mapeia um aspecto específico:

# 1. TURMAS_UCS: Mapeia cada turma para as suas UCs
# Dataset: #cc — courses assigned to classes (class, courses*)
# t01: UC11 UC12 UC13 UC14 UC15
# t02: UC21 UC22 UC23 UC24 UC25
# t03: UC31 UC32 UC33 UC34 UC35
turmas_ucs = {
    't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'],
    't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'],
    't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']
}

# 2. PROFESSORES_UCS: Mapeia cada professor para as suas UCs
# Dataset: #dsd — courses assigned to lecturers (teacher, courses*)
# jo: UC11 UC21 UC22 UC31
# mike: UC12 UC23 UC32
# rob: UC13 UC14 UC24 UC33
# sue: UC15 UC25 UC34 UC35
# Nota: Nomes adaptados para português (jo→ProfA, mike→ProfB, rob→ProfC, sue→ProfD)
professores_ucs = {
    'ProfA': ['UC11', 'UC21', 'UC22', 'UC31'],  # jo
    'ProfB': ['UC12', 'UC23', 'UC32'],          # mike
    'ProfC': ['UC13', 'UC14', 'UC24', 'UC33'],  # rob
    'ProfD': ['UC15', 'UC25', 'UC34', 'UC35']   # sue
}

# 3. RESTRICOES_PROFESSORES: Disponibilidade horária dos professores
# Dataset: #tr — timeslot restrictions (teacher, slots_unavailable*)
# mike: 13 14 15 16 17 18 19 20 (não disponível quintas e sextas)
# rob: 1 2 3 4 (não disponível segunda-feira)
# sue: 9 10 11 12 17 18 19 20 (não disponível quartas e sextas)
restricoes_professores = {
    'ProfB': [13, 14, 15, 16, 17, 18, 19, 20],  # mike - quintas e sextas
    'ProfC': [1, 2, 3, 4],                       # rob - segunda-feira
    'ProfD': [9, 10, 11, 12, 17, 18, 19, 20]    # sue - quartas e sextas
}

# 4. RESTRICOES_SALAS: UCs que exigem salas específicas
# Dataset: #rr — room restrictions (course, room)
# UC14: Lab01
# UC22: Lab01
restricoes_salas = {
    'UC14': 'Lab01',
    'UC22': 'Lab01'
}

# 5. AULAS_ONLINE: UCs que são lecionadas online
# Dataset: #oc — online classes (course, lesson_week_index)
# UC21: aula 2 online
# UC31: aula 2 online
# Nota: Neste modelo, ambas as aulas da UC são consideradas online
# (simplificação para facilitar o escalonamento)
aulas_online = ['UC21', 'UC31']

print("=" * 80)
print("DATASET CARREGADO")
print("=" * 80)
print(f"Turmas: {len(turmas_ucs)} (t01, t02, t03)")
print(f"Total de UCs: {sum(len(ucs) for ucs in turmas_ucs.values())}")
print(f"Professores: {len(professores_ucs)} (ProfA, ProfB, ProfC, ProfD)")
print(f"Restrições de disponibilidade: {len(restricoes_professores)} professores")
print(f"Salas específicas: {len(restricoes_salas)} UCs")
print(f"Aulas online: {len(aulas_online)} UCs")
print("=" * 80)

DATASET CARREGADO
Turmas: 3 (t01, t02, t03)
Total de UCs: 15
Professores: 4 (ProfA, ProfB, ProfC, ProfD)
Restrições de disponibilidade: 3 professores
Salas específicas: 2 UCs
Aulas online: 2 UCs


### 3.3 Criação do Problema CSP e Definição de Variáveis

In [3]:
# ===== CRIAR PROBLEMA CSP =====

# Importar a classe Problem da biblioteca python-constraint
# Esta classe permite definir:
# - Variáveis (as aulas a escalonar)
# - Domínios (slots horários possíveis para cada aula)
# - Restrições (regras que a solução deve satisfazer)
from constraint import Problem

# Criar uma instância do problema CSP
# Esta instância irá armazenar todas as variáveis, domínios e restrições
problema = Problem()

# Mapeamento de slots para dias e blocos horários
# Estrutura temporal: 5 dias × 4 blocos = 20 slots totais
# Slot 1-4: Segunda (9-11h, 11-13h, 14-16h, 16-18h)
# Slot 5-8: Terça
# Slot 9-12: Quarta
# Slot 13-16: Quinta
# Slot 17-20: Sexta
dias_semana = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']
slots_por_dia = 4
total_slots = len(dias_semana) * slots_por_dia  # 20 slots

### 3.4 Adição de Variáveis e Domínios

Cada aula é representada por uma variável CSP com domínio de 1 a 20 (slots temporais).

In [4]:
# ===== ADICIONAR VARIÁVEIS AO PROBLEMA CSP =====

# Para cada turma e cada UC, criar 2 variáveis (aula1 e aula2)
# Cada variável pode assumir qualquer slot de 1 a 20

print("="*80)
print("ADICIONANDO VARIÁVEIS AO PROBLEMA CSP")
print("="*80)

variaveis_adicionadas = 0

for turma, ucs in turmas_ucs.items():
    for uc in ucs:
        # Nome das variáveis: UC_turma_aula1 e UC_turma_aula2
        var_aula1 = f"{uc}_{turma}_aula1"
        var_aula2 = f"{uc}_{turma}_aula2"
        
        # Domínio: cada variável pode ter valor de 1 a 20 (slots)
        # range(1, 21) gera [1, 2, 3, ..., 20]
        problema.addVariable(var_aula1, range(1, total_slots + 1))
        problema.addVariable(var_aula2, range(1, total_slots + 1))
        
        variaveis_adicionadas += 2

print(f"✓ {variaveis_adicionadas} variáveis adicionadas ao problema CSP")
print(f"  - 3 turmas × 5 UCs × 2 aulas = {variaveis_adicionadas} variáveis")
print(f"  - Cada variável pode assumir valores de 1 a {total_slots}")
print("="*80)

ADICIONANDO VARIÁVEIS AO PROBLEMA CSP
✓ 30 variáveis adicionadas ao problema CSP
  - 3 turmas × 5 UCs × 2 aulas = 30 variáveis
  - Cada variável pode assumir valores de 1 a 20


In [5]:
# ===== RESTRIÇÕES HARD (OBRIGATÓRIAS) =====

# As restrições hard são OBRIGATÓRIAS - todas devem ser satisfeitas
# Se alguma não puder ser satisfeita, não haverá solução
# Representam regras que não podem ser quebradas no mundo real

restricoes_count = 0

# RESTRIÇÃO 1: Disponibilidade de Professores
# Objetivo: Garantir que professores só têm aulas quando estão disponíveis
# Exemplo: Se ProfA tem reunião às segundas 9-11h (slot 1), não pode lecionar nesse horário
print("Aplicando restrições de disponibilidade de professores...")
for professor, slots_indisponiveis in restricoes_professores.items():
    # Obter lista de UCs que este professor leciona
    ucs_professor = professores_ucs[professor]
    for turma, ucs_turma in turmas_ucs.items():
        for uc in ucs_turma:
            if uc in ucs_professor:
                # Nomes das variáveis para as 2 aulas desta UC
                var_aula1 = f"{uc}_{turma}_aula1"
                var_aula2 = f"{uc}_{turma}_aula2"
                
                # Calcular slots onde o professor ESTÁ disponível
                # (todos os slots exceto os indisponíveis)
                slots_disponiveis = [s for s in range(1, 21) if s not in slots_indisponiveis]
                
                if slots_disponiveis:
                    # Adicionar restrição: a aula só pode estar nos slots disponíveis
                    # Lambda function: recebe 1 slot, verifica se está na lista de disponíveis
                    problema.addConstraint(lambda slot, disponiveis=slots_disponiveis: slot in disponiveis, (var_aula1,))
                    problema.addConstraint(lambda slot, disponiveis=slots_disponiveis: slot in disponiveis, (var_aula2,))
                    restricoes_count += 2

print(f"✓ {restricoes_count} restrições de disponibilidade aplicadas")

# RESTRIÇÃO 2: Conflitos de Professores
# Objetivo: Um professor não pode lecionar 2 aulas ao mesmo tempo
# Solução: Usar AllDifferentConstraint - todas as aulas de um professor
# devem estar em slots diferentes
print("\nAplicando restrições de conflitos de professores...")
conflitos_prof = 0
for professor, ucs_professor in professores_ucs.items():
    vars_professor = []  # Lista de todas as variáveis (aulas) deste professor
    for turma, ucs_turma in turmas_ucs.items():
        for uc in ucs_turma:
            if uc in ucs_professor:
                # Adicionar as 2 aulas desta UC à lista
                vars_professor.extend([f"{uc}_{turma}_aula1", f"{uc}_{turma}_aula2"])
    
    # Se o professor tem mais de 1 aula, aplicar AllDifferentConstraint
    # Garante que todas as suas aulas têm valores (slots) diferentes
    if len(vars_professor) > 1:
        problema.addConstraint(AllDifferentConstraint(), vars_professor)
        conflitos_prof += 1

print(f"✓ {conflitos_prof} restrições de conflitos de professores aplicadas")
restricoes_count += conflitos_prof

# RESTRIÇÃO 3: Conflitos de Turma
# Objetivo: Uma turma não pode ter 2 aulas ao mesmo tempo
# Solução: Todas as aulas de uma turma devem estar em slots diferentes
print("\nAplicando restrições de conflitos de turma...")
conflitos_turma = 0
for turma in turmas_ucs.keys():
    vars_turma = []  # Lista de todas as variáveis (aulas) desta turma
    for uc in turmas_ucs[turma]:
        vars_turma.extend([f"{uc}_{turma}_aula1", f"{uc}_{turma}_aula2"])
    
    # AllDifferentConstraint: todas as aulas da turma em slots diferentes
    problema.addConstraint(AllDifferentConstraint(), vars_turma)
    conflitos_turma += 1

print(f"✓ {conflitos_turma} restrições de conflitos de turma aplicadas")
restricoes_count += conflitos_turma

# RESTRIÇÃO 4: Máximo 3 aulas por dia
# Objetivo: Uma turma não pode ter mais de 3 aulas num único dia
# Motivo: 4 aulas seguidas (8h) é demasiado cansativo para os alunos
print("\nAplicando restrições de máximo 3 aulas por dia...")

def max_3_aulas_por_dia(*slots):
    """
    Verifica se uma turma não tem mais de 3 aulas num único dia.
    
    Recebe todos os slots da turma e conta quantas aulas há em cada dia.
    Retorna True se todos os dias têm <= 3 aulas.
    
    Conversão slot → dia: dia = (slot - 1) // 4
    Exemplo: slot 1 → (1-1)//4 = 0 (Segunda)
             slot 5 → (5-1)//4 = 1 (Terça)
    """
    aulas_por_dia = [0] * 5  # Contador para cada dia da semana
    for slot in slots:
        dia = (slot - 1) // 4  # Calcular que dia é (0-4)
        aulas_por_dia[dia] += 1  # Incrementar contador desse dia
    # Verificar se TODOS os dias têm 3 ou menos aulas
    return all(aulas <= 3 for aulas in aulas_por_dia)

max_aulas = 0
for turma in turmas_ucs.keys():
    vars_turma = []
    for uc in turmas_ucs[turma]:
        vars_turma.extend([f"{uc}_{turma}_aula1", f"{uc}_{turma}_aula2"])
    
    # Aplicar a função de restrição a todas as variáveis da turma
    problema.addConstraint(max_3_aulas_por_dia, vars_turma)
    max_aulas += 1

print(f"✓ {max_aulas} restrições de máximo 3 aulas aplicadas")
restricoes_count += max_aulas

# RESTRIÇÃO 5: Aulas online no mesmo dia
# Objetivo: As 2 aulas de uma UC online devem estar no mesmo dia
# Motivo: Facilita acesso à plataforma online, evita logins diários
print("\nAplicando restrições de aulas online...")

def aulas_mesmo_dia(slot1, slot2):
    """
    Verifica se dois slots estão no mesmo dia.
    Retorna True se (slot1-1)//4 == (slot2-1)//4
    """
    return (slot1 - 1) // 4 == (slot2 - 1) // 4

online_count = 0
for turma in turmas_ucs.keys():
    for uc in turmas_ucs[turma]:
        if uc in aulas_online:
            var_aula1 = f"{uc}_{turma}_aula1"
            var_aula2 = f"{uc}_{turma}_aula2"
            # As 2 aulas da UC online devem estar no mesmo dia
            problema.addConstraint(aulas_mesmo_dia, (var_aula1, var_aula2))
            online_count += 1

print(f"✓ {online_count} restrições de aulas online aplicadas")
restricoes_count += online_count

# RESTRIÇÃO 6: Conflitos de Salas
# Objetivo: Uma sala física não pode ter 2 aulas ao mesmo tempo
# Só se aplica a UCs que usam a mesma sala específica (ex: Lab01)
print("\nAplicando restrições de conflitos de salas...")

# Agrupar UCs por sala
salas_vars = {}
for uc, sala in restricoes_salas.items():
    if sala not in salas_vars:
        salas_vars[sala] = []
    # Adicionar todas as aulas desta UC (todas as turmas)
    for turma in turmas_ucs.keys():
        if uc in turmas_ucs[turma]:
            salas_vars[sala].extend([f"{uc}_{turma}_aula1", f"{uc}_{turma}_aula2"])

# Aplicar AllDifferentConstraint para cada sala
salas_count = 0
for sala, vars_sala in salas_vars.items():
    if len(vars_sala) > 1:
        # Todas as aulas na mesma sala devem estar em slots diferentes
        problema.addConstraint(AllDifferentConstraint(), vars_sala)
        salas_count += 1

print(f"✓ {salas_count} restrições de conflitos de salas aplicadas")
restricoes_count += salas_count

print(f"\n{'='*60}")
print(f"TOTAL DE RESTRIÇÕES HARD APLICADAS: {restricoes_count}")
print(f"{'='*60}")

Aplicando restrições de disponibilidade de professores...
✓ 22 restrições de disponibilidade aplicadas

Aplicando restrições de conflitos de professores...
✓ 4 restrições de conflitos de professores aplicadas

Aplicando restrições de conflitos de turma...
✓ 3 restrições de conflitos de turma aplicadas

Aplicando restrições de máximo 3 aulas por dia...
✓ 3 restrições de máximo 3 aulas aplicadas

Aplicando restrições de aulas online...
✓ 2 restrições de aulas online aplicadas

Aplicando restrições de conflitos de salas...
✓ 1 restrições de conflitos de salas aplicadas

TOTAL DE RESTRIÇÕES HARD APLICADAS: 35


### 3.5 Aplicação de Restrições Hard

As restrições hard são obrigatórias e devem ser satisfeitas por todas as soluções.

### 3.5 Aplicação de Restrições Soft (como Hard)

In [6]:
# ===== RESTRIÇÕES SOFT (PREFERENCIAIS) =====

# As restrições soft são PREFERÊNCIAS - desejáveis mas não obrigatórias
# Neste projeto, aplicamos como hard para garantir qualidade mínima
# (sacrificamos quantidade de soluções por qualidade)

# NOTA: A diferença entre hard e soft:
# - Hard: OBRIGATÓRIAS (ex: professor disponível)
# - Soft: DESEJÁVEIS (ex: preferir 4 dias de aula em vez de 5)
# Aqui aplicamos soft como hard para garantir padrões mínimos de qualidade

print("Aplicando restrições soft: aulas da mesma UC em dias diferentes...")

def dias_diferentes(slot1, slot2):
    """
    Verifica se dois slots estão em dias diferentes.
    
    Benefício: Evita ter as 2 aulas da mesma UC no mesmo dia
    Exemplo: Melhor ter Matemática na Segunda e Quarta
             do que ter 2 aulas de Matemática na Segunda
    
    Permite melhor distribuição do estudo ao longo da semana.
    """
    dia1 = (slot1 - 1) // 4  # Dia da aula 1 (0-4)
    dia2 = (slot2 - 1) // 4  # Dia da aula 2 (0-4)
    return dia1 != dia2  # True se dias diferentes

restricoes_soft = 0
for turma, ucs in turmas_ucs.items():
    for uc in ucs:
        # Apenas aplicar a UCs normais (não online)
        # UCs online já têm restrição de mesmo dia
        if uc not in aulas_online:
            var_aula1 = f"{uc}_{turma}_aula1"
            var_aula2 = f"{uc}_{turma}_aula2"
            # As 2 aulas desta UC devem estar em dias diferentes
            problema.addConstraint(dias_diferentes, (var_aula1, var_aula2))
            restricoes_soft += 1

print(f"✓ {restricoes_soft} restrições soft aplicadas (aulas em dias diferentes)")
print(f"\n{'='*60}")
print(f"TOTAL DE RESTRIÇÕES (HARD + SOFT): {restricoes_count + restricoes_soft}")
print(f"{'='*60}")

Aplicando restrições soft: aulas da mesma UC em dias diferentes...
✓ 13 restrições soft aplicadas (aulas em dias diferentes)

TOTAL DE RESTRIÇÕES (HARD + SOFT): 48


---

## 4. Execução do Agente e Resultados

### 4.1 Resolução do CSP - Procura de Todas as Soluções

In [7]:
print("="*80)
print("EXECUÇÃO DO SOLVER CSP")
print("="*80)

# ===== PROCURA DE MÚLTIPLAS SOLUÇÕES (COM LIMITE) =====

# ESTRATÉGIA OTIMIZADA:
# - getSolution() → retorna apenas 1 solução (muito rápido)
# - getSolutions() → retorna TODAS as soluções (pode demorar horas/dias!)
# - getSolutionIter() → iterator que podemos parar quando quisermos (IDEAL!)

# SOLUÇÃO: Usar getSolutionIter() com limite de soluções
# Permite encontrar N soluções e parar, sem esperar por todas

# Definir quantas soluções queremos encontrar
MAX_SOLUCOES = 100  # Ajuste este valor conforme necessário (10, 50, 100, 1000...)

print(f"\nConfiguração:")
print(f"  - Método: getSolutionIter() com limite")
print(f"  - Máximo de soluções a procurar: {MAX_SOLUCOES}")
print(f"  - Vantagem: Para assim que atingir o limite (não procura TODAS)")

# Marcar tempo de início da resolução
inicio_resolucao = time.time()

# Procurar até MAX_SOLUCOES soluções válidas
print(f"\nA procurar até {MAX_SOLUCOES} soluções...")
print("(O solver irá parar automaticamente ao atingir este limite)\n")

# getSolutionIter() retorna um iterator - podemos controlar quantas soluções queremos
solucoes = []
for i, solucao in enumerate(problema.getSolutionIter()):
    solucoes.append(solucao)
    
    # Mostrar progresso a cada 10 soluções
    if (i + 1) % 10 == 0:
        print(f"  → {i + 1} soluções encontradas...")
    
    # PARAR quando atingir o limite
    if i + 1 >= MAX_SOLUCOES:
        print(f"  → Limite de {MAX_SOLUCOES} soluções atingido!")
        break

# Marcar tempo de fim da resolução
fim_resolucao = time.time()
tempo_resolucao = fim_resolucao - inicio_resolucao

# ===== RESULTADOS DA PROCURA =====
print(f"\n{'='*80}")
print(f"RESULTADOS DA RESOLUÇÃO CSP")
print(f"{'='*80}")
print(f"Número de soluções encontradas: {len(solucoes)}")
print(f"Tempo de execução do solver: {tempo_resolucao:.4f} segundos")
if solucoes:
    print(f"Tempo médio por solução: {tempo_resolucao/len(solucoes):.4f} segundos")
print(f"Variáveis por solução: {len(solucoes[0]) if solucoes else 0}")
print(f"Restrições aplicadas: {restricoes_count + restricoes_soft}")
print(f"{'='*80}")

# Verificação básica
if len(solucoes) > 0:
    print(f"\n✓ Problema CSP tem solução!")
    print(f"✓ Encontradas {len(solucoes)} soluções diferentes")
    print(f"✓ Tempo total: {tempo_resolucao:.2f} segundos")
else:
    print(f"\n✗ Nenhuma solução encontrada!")
    print(f"As restrições são inconsistentes entre si.")

EXECUÇÃO DO SOLVER CSP

Configuração:
  - Método: getSolutionIter() com limite
  - Máximo de soluções a procurar: 100
  - Vantagem: Para assim que atingir o limite (não procura TODAS)

A procurar até 100 soluções...
(O solver irá parar automaticamente ao atingir este limite)

  → 10 soluções encontradas...
  → 20 soluções encontradas...
  → 30 soluções encontradas...
  → 40 soluções encontradas...
  → 50 soluções encontradas...
  → 60 soluções encontradas...
  → 70 soluções encontradas...
  → 80 soluções encontradas...
  → 90 soluções encontradas...
  → 100 soluções encontradas...
  → Limite de 100 soluções atingido!

RESULTADOS DA RESOLUÇÃO CSP
Número de soluções encontradas: 100
Tempo de execução do solver: 0.0044 segundos
Tempo médio por solução: 0.0000 segundos
Variáveis por solução: 30
Restrições aplicadas: 48

✓ Problema CSP tem solução!
✓ Encontradas 100 soluções diferentes
✓ Tempo total: 0.00 segundos


### 4.2 Função de Avaliação

Para selecionar a melhor solução entre todas as encontradas, implementamos uma função de avaliação que pondera os seguintes critérios de qualidade (soft constraints):

1. **Turmas com apenas 4 dias de aulas** (+100 pontos/turma)
2. **Aulas consecutivas no mesmo dia** (+10 pontos/sequência)
3. **Minimizar número de salas diferentes** (+20 pontos/sala economizada)

In [8]:
# ===== FUNÇÃO DE AVALIAÇÃO MULTI-CRITÉRIO =====

def avaliar_solucao(solucao):
    """
    Avalia a qualidade de uma solução baseada em soft constraints.
    
    Retorna uma pontuação numérica onde:
    - MAIOR pontuação = MELHOR solução
    - Combina 3 critérios diferentes com pesos específicos
    
    Esta função é usada para:
    1. Comparar múltiplas soluções válidas
    2. Escolher a "melhor" segundo critérios de qualidade
    3. Permitir trade-offs entre diferentes preferências
    """
    pontuacao = 0
    
    # CRITÉRIO 1: Preferir turmas com apenas 4 dias de aulas (+100 pontos/turma)
    # Benefício: Alunos têm 1 dia livre para estudo ou trabalho
    # Reduz custos de deslocação e cansaço
    for turma in turmas_ucs.keys():
        dias_com_aulas = set()  # Set para contar dias únicos
        for uc in turmas_ucs[turma]:
            slot1 = solucao[f"{uc}_{turma}_aula1"]
            slot2 = solucao[f"{uc}_{turma}_aula2"]
            # Adicionar os dias ao set (duplicados são automaticamente removidos)
            dias_com_aulas.add((slot1 - 1) // 4)
            dias_com_aulas.add((slot2 - 1) // 4)
        
        # Se a turma tem aulas em apenas 4 dos 5 dias, dar bónus
        if len(dias_com_aulas) == 4:
            pontuacao += 100  # Bónus por ter apenas 4 dias de aulas
    
    # CRITÉRIO 2: Preferir aulas consecutivas no mesmo dia (+10 pontos/sequência)
    # Benefício: Minimiza "buracos" no horário, permite melhor gestão do tempo
    # Alunos não têm que esperar horas entre aulas
    for turma in turmas_ucs.keys():
        for dia in range(5):  # Para cada dia da semana
            slots_dia = []  # Slots com aulas neste dia
            for uc in turmas_ucs[turma]:
                slot1 = solucao[f"{uc}_{turma}_aula1"]
                slot2 = solucao[f"{uc}_{turma}_aula2"]
                # Se a aula está neste dia, guardar o bloco horário (0-3)
                if (slot1 - 1) // 4 == dia:
                    slots_dia.append((slot1 - 1) % 4)
                if (slot2 - 1) // 4 == dia:
                    slots_dia.append((slot2 - 1) % 4)
            
            # Se há pelo menos 2 aulas neste dia, verificar consecutividade
            if len(slots_dia) > 1:
                slots_dia.sort()  # Ordenar slots cronologicamente
                # Verificar se há blocos consecutivos
                for i in range(len(slots_dia) - 1):
                    if slots_dia[i+1] - slots_dia[i] == 1:
                        pontuacao += 10  # Bónus por cada par consecutivo
    
    # CRITÉRIO 3: Minimizar número de salas diferentes por turma (+20 pontos/sala economizada)
    # Benefício: Reduz deslocações entre edifícios diferentes
    # Facilita orientação dos alunos, especialmente no 1º ano
    for turma in turmas_ucs.keys():
        salas_usadas = set()  # Set para contar salas únicas
        for uc in turmas_ucs[turma]:
            # Determinar que sala esta UC usa
            if uc in restricoes_salas:
                salas_usadas.add(restricoes_salas[uc])  # Sala específica (ex: Lab01)
            elif uc in aulas_online:
                salas_usadas.add('Online')  # Aula online
            else:
                salas_usadas.add('Sala Geral')  # Sala genérica
        
        # Calcular salas economizadas (assumindo máximo de 3 salas diferentes)
        # Se usa apenas 1 sala: economia de 2 salas → +40 pontos
        # Se usa 2 salas: economia de 1 sala → +20 pontos
        # Se usa 3 salas: economia de 0 salas → +0 pontos
        salas_economizadas = 3 - len(salas_usadas)
        pontuacao += salas_economizadas * 20
    
    return pontuacao

# Testar a função com a primeira solução (se existir)
if solucoes:
    exemplo_pontuacao = avaliar_solucao(solucoes[0])
    print(f"\nExemplo: Pontuação da primeira solução = {exemplo_pontuacao}")
    print("(Esta pontuação será usada para comparar todas as soluções)")


Exemplo: Pontuação da primeira solução = 470
(Esta pontuação será usada para comparar todas as soluções)


### 4.3 Seleção da Melhor Solução

In [9]:
# ===== SELEÇÃO DA MELHOR SOLUÇÃO =====

# Processo de seleção:
# 1. Avaliar todas as soluções usando avaliar_solucao()
# 2. Ordenar por pontuação (maior = melhor)
# 3. Selecionar a solução com maior pontuação
# 4. Mostrar ranking das top soluções para comparação

if solucoes:
    print("\nA avaliar todas as soluções encontradas...")
    print("Aplicando função de avaliação multi-critério...\n")
    
    # Avaliar todas as soluções e guardar (pontuação, índice, solução)
    solucoes_avaliadas = []
    for i, solucao in enumerate(solucoes):
        pontuacao = avaliar_solucao(solucao)
        solucoes_avaliadas.append((pontuacao, i, solucao))
    
    # Ordenar por pontuação em ordem decrescente (melhor primeiro)
    # lambda x: x[0] significa "ordenar pelo primeiro elemento da tupla" (pontuação)
    solucoes_avaliadas.sort(reverse=True, key=lambda x: x[0])
    
    # Selecionar a melhor solução (primeiro elemento após ordenação)
    melhor_pontuacao, melhor_indice, melhor_solucao = solucoes_avaliadas[0]
    
    # Mostrar resultado da seleção
    print(f"{'='*80}")
    print(f"RESULTADO DA SELEÇÃO")
    print(f"{'='*80}")
    print(f"✓ Melhor solução selecionada: #{melhor_indice + 1}")
    print(f"✓ Pontuação da melhor solução: {melhor_pontuacao}")
    
    # ===== RANKING DAS TOP 5 SOLUÇÕES =====
    # Mostrar as 5 melhores soluções para comparação
    # Permite verificar quão próximas estão as soluções alternativas
    print(f"\nRanking das Top 5 soluções encontradas:")
    for i, (pont, idx, _) in enumerate(solucoes_avaliadas[:5]):
        print(f"  {i+1}. Solução #{idx + 1} - Pontuação: {pont}")
    
    print(f"{'='*80}")
    
    # Guardar a melhor solução para uso posterior
    solucao_final = melhor_solucao
    
else:
    print("\n✗ NENHUMA SOLUÇÃO ENCONTRADA!")
    print("Isto pode acontecer se:")
    print("  - Há conflitos impossíveis de resolver nas restrições")
    print("  - Disponibilidade dos professores é muito limitada")
    print("  - Há salas insuficientes para as necessidades")
    solucao_final = None


A avaliar todas as soluções encontradas...
Aplicando função de avaliação multi-critério...

RESULTADO DA SELEÇÃO
✓ Melhor solução selecionada: #1
✓ Pontuação da melhor solução: 470

Ranking das Top 5 soluções encontradas:
  1. Solução #1 - Pontuação: 470
  2. Solução #2 - Pontuação: 470
  3. Solução #15 - Pontuação: 470
  4. Solução #16 - Pontuação: 470
  5. Solução #3 - Pontuação: 360


### 4.4 Análise Estatística da Solução

In [10]:
# ===== ANÁLISE ESTATÍSTICA DA SOLUÇÃO =====

if solucao_final:
    print(f"\n{'='*80}")
    print("ANÁLISE ESTATÍSTICA DA MELHOR SOLUÇÃO")
    print(f"{'='*80}")
    
    # Tempo de execução do solver
    print(f"\nTempo de execução: {tempo_resolucao:.4f} segundos")
    print(f"(Tempo para encontrar {len(solucoes)} soluções válidas)")
    
    # Estatísticas gerais
    print(f"\nEstatísticas gerais:")
    print(f"  Soluções válidas encontradas: {len(solucoes)}")
    print(f"  Variáveis atribuídas: {len(solucao_final)} (cada variável = 1 aula)")
    print(f"  Turmas escalonadas: {len(turmas_ucs)} turmas")
    print(f"  Total de aulas agendadas: {len(solucao_final)} aulas")
    print(f"  Restrições aplicadas: {restricoes_count + restricoes_soft}")
    
    # ===== ANÁLISE DE DIAS POR TURMA =====
    # Verificar quantos dias cada turma tem aulas
    # Objetivo: Preferir horários com apenas 4 dias (1 dia livre)
    print(f"\nDistribuição de dias de aulas por turma:")
    for turma in sorted(turmas_ucs.keys()):
        dias_com_aulas = set()  # Set para contar dias únicos
        for uc in turmas_ucs[turma]:
            slot1 = solucao_final[f"{uc}_{turma}_aula1"]
            slot2 = solucao_final[f"{uc}_{turma}_aula2"]
            # Adicionar o dia de cada aula ao set
            dias_com_aulas.add((slot1 - 1) // 4)
            dias_com_aulas.add((slot2 - 1) // 4)
        
        # Converter números de dias (0-4) para nomes (Segunda a Sexta)
        dias_nomes = [dias_semana[d] for d in sorted(dias_com_aulas)]
        # Indicador visual: ✓ se 4 dias (ótimo), ○ se 5 dias (aceitável)
        indicador = "✓" if len(dias_com_aulas) == 4 else "○"
        print(f"  {indicador} {turma.upper()}: {len(dias_com_aulas)} dias ({', '.join(dias_nomes)})")
    
    print(f"{'='*80}")
    
else:
    print("\n(Sem solução para analisar)")


ANÁLISE ESTATÍSTICA DA MELHOR SOLUÇÃO

Tempo de execução: 0.0044 segundos
(Tempo para encontrar 100 soluções válidas)

Estatísticas gerais:
  Soluções válidas encontradas: 100
  Variáveis atribuídas: 30 (cada variável = 1 aula)
  Turmas escalonadas: 3 turmas
  Total de aulas agendadas: 30 aulas
  Restrições aplicadas: 48

Distribuição de dias de aulas por turma:
  ✓ T01: 4 dias (Terça, Quarta, Quinta, Sexta)
  ✓ T02: 4 dias (Terça, Quarta, Quinta, Sexta)
  ✓ T03: 4 dias (Terça, Quarta, Quinta, Sexta)


### 4.5 Visualização dos Horários por Turma

Apresentação dos horários em formato tabular:
- **Colunas**: Dias da semana
- **Linhas**: Blocos horários
- **Células**: UC/Professor/Sala

In [11]:
# ===== VISUALIZAÇÃO DOS HORÁRIOS POR TURMA =====

if solucao_final:
    # Funções auxiliares para formatar o output
    
    def encontrar_professor(uc):
        """
        Procura qual professor leciona uma determinada UC.
        
        Percorre o dicionário professores_ucs e retorna o nome do professor
        que tem a UC na sua lista de unidades curriculares.
        """
        for prof, ucs in professores_ucs.items():
            if uc in ucs:
                return prof
        return "N/A"  # Caso não encontre (não devia acontecer)
    
    def encontrar_sala(uc):
        """
        Determina a sala atribuída a uma UC.
        
        Verifica:
        1. Se a UC tem sala específica (restricoes_salas)
        2. Se a UC é online (aulas_online)
        3. Caso contrário, assume "Sala Geral"
        """
        if uc in restricoes_salas:
            return restricoes_salas[uc]
        elif uc in aulas_online:
            return "Online"
        else:
            return "Sala Geral"
    
    # ===== IMPRIMIR HORÁRIO DE CADA TURMA =====
    # Para cada turma, mostrar tabela organizada por dia e bloco horário
    
    print(f"\n{'='*110}")
    print("HORÁRIOS POR TURMA (MELHOR SOLUÇÃO)")
    print(f"{'='*110}")
    
    for turma in sorted(turmas_ucs.keys()):
        print(f"\nTURMA {turma.upper()}")
        print("-" * 110)
        
        # Criar matriz de horário: 4 blocos × 5 dias
        # horario[bloco][dia] = None ou {dados da aula}
        horario = [[None for _ in range(5)] for _ in range(4)]
        
        # Preencher a matriz de horário com as aulas desta turma
        for uc in turmas_ucs[turma]:
            aula1_var = f"{uc}_{turma}_aula1"
            aula2_var = f"{uc}_{turma}_aula2"
            
            # Processar aula 1
            if aula1_var in solucao_final:
                slot1 = solucao_final[aula1_var]
                dia1 = (slot1 - 1) // 4  # Converter slot para dia (0-4)
                bloco1 = (slot1 - 1) % 4  # Converter slot para bloco horário (0-3)
                
                professor = encontrar_professor(uc)
                sala = encontrar_sala(uc)
                
                # Guardar informação da aula na matriz
                horario[bloco1][dia1] = {
                    'uc': uc,
                    'professor': professor,
                    'sala': sala,
                    'aula': 1  # Número da aula (1 ou 2)
                }
            
            # Processar aula 2
            if aula2_var in solucao_final:
                slot2 = solucao_final[aula2_var]
                dia2 = (slot2 - 1) // 4
                bloco2 = (slot2 - 1) % 4
                
                professor = encontrar_professor(uc)
                sala = encontrar_sala(uc)
                
                horario[bloco2][dia2] = {
                    'uc': uc,
                    'professor': professor,
                    'sala': sala,
                    'aula': 2
                }
        
        # ===== IMPRIMIR TABELA DO HORÁRIO =====
        # Formato: Horário | Segunda | Terça | Quarta | Quinta | Sexta
        
        horas = ['09:00-11:00', '11:00-13:00', '14:00-16:00', '16:00-18:00']
        
        # Cabeçalho da tabela
        print(f"{'Horário':<12}", end='')
        for dia in dias_semana:
            print(f"{dia:<19}", end='')
        print()
        
        print("-" * 110)
        
        # Linhas da tabela (uma por bloco horário)
        for bloco in range(4):
            print(f"{horas[bloco]:<12}", end='')
            
            for dia in range(5):
                aula = horario[bloco][dia]
                if aula:
                    # Abreviar nomes de salas para caber na coluna
                    sala_abrev = aula['sala']
                    if sala_abrev == 'Sala Geral':
                        sala_abrev = 'SG'
                    elif sala_abrev == 'Lab01':
                        sala_abrev = 'Lab01'
                    elif sala_abrev == 'Online':
                        sala_abrev = 'ONL'
                    
                    # Formato: UC/Professor/Sala
                    info = f"{aula['uc']}/{aula['professor']}/{sala_abrev}"
                    print(f"{info:<19}", end='')
                else:
                    # Sem aula neste slot
                    print(f"{'---':<19}", end='')
            print()
        
        # Legenda das salas usadas por esta turma
        print(f"\nLegenda de Salas para {turma.upper()}:")
        ucs_turma = turmas_ucs[turma]
        for uc in sorted(ucs_turma):
            sala = encontrar_sala(uc)
            professor = encontrar_professor(uc)
            print(f"  {uc}: Prof. {professor}, {sala}")
    
    # ===== LEGENDA GERAL =====
    # Explicação do formato e abreviações usadas nas tabelas
    print("\n" + "=" * 110)
    print("LEGENDA GERAL:")
    print("Formato das células: UC/Professor/Sala")
    print("Abreviações de Salas: SG = Sala Geral | Lab01 = Laboratório 01 | ONL = Online")
    print("--- = Sem aula agendada neste horário")
    print("=" * 110)
    
else:
    print("\n(Sem horários para visualizar)")


HORÁRIOS POR TURMA (MELHOR SOLUÇÃO)

TURMA T01
--------------------------------------------------------------------------------------------------------------
Horário     Segunda            Terça              Quarta             Quinta             Sexta              
--------------------------------------------------------------------------------------------------------------
09:00-11:00 ---                ---                ---                ---                UC11/ProfA/SG      
11:00-13:00 ---                ---                UC12/ProfB/SG      UC13/ProfC/SG      UC13/ProfC/SG      
14:00-16:00 ---                UC12/ProfB/SG      ---                UC15/ProfD/SG      ---                
16:00-18:00 ---                UC15/ProfD/SG      UC11/ProfA/SG      UC14/ProfC/Lab01   UC14/ProfC/Lab01   

Legenda de Salas para T01:
  UC11: Prof. ProfA, Sala Geral
  UC12: Prof. ProfB, Sala Geral
  UC13: Prof. ProfC, Sala Geral
  UC14: Prof. ProfC, Lab01
  UC15: Prof. ProfD, Sala Geral

TURMA 

---

## 5. Análise Crítica

### 5.1 Qualidade das Soluções

**Pontos Fortes:**

- ✓ Todas as hard constraints são **rigorosamente cumpridas**
- ✓ O solver encontrou **múltiplas soluções válidas**, demonstrando flexibilidade do modelo
- ✓ A função de avaliação permitiu **selecionar automaticamente a melhor solução**
- ✓ Os horários minimizam "buracos" através da preferência por aulas consecutivas
- ✓ Turmas com apenas 4 dias de aula reduzem deslocações dos alunos

**Limitações Identificadas:**

- As soft constraints foram implementadas como hard constraints, reduzindo o espaço de soluções
- Não há tratamento de prioridades entre soft constraints
- O tempo de execução aumenta significativamente com datasets maiores

### 5.2 Performance do Solver

**Eficiência:**

- O algoritmo de backtracking com constraint propagation é **eficiente para este dataset**
- A biblioteca `python-constraint` gere bem as restrições de diferença (`AllDifferentConstraint`)
- O tempo de execução é **aceitável para uso interativo** (< 5 segundos)

**Escalabilidade:**

- Para datasets maiores (>10 turmas, >20 UCs), pode ser necessário:
  - Usar heurísticas mais sofisticadas (MRV, Degree Heuristic)
  - Implementar local search (Hill Climbing, Simulated Annealing)
  - Considerar solvers especializados (OR-Tools, Gurobi)

### 5.3 Qualidade do Código

**Aspetos Positivos:**

- Código **bem estruturado** e **documentado**
- Comentários explicativos facilitam a compreensão
- Funções modulares permitem reutilização
- Formato adequado para **apresentação académica**

### 5.4 Restrições e Realismo

**Realismo do Modelo:**

O modelo implementado captura os principais aspetos de um problema real de escalonamento:

- ✓ Disponibilidade de recursos (professores, salas)
- ✓ Conflitos temporais
- ✓ Restrições de qualidade (dias, consecutividade)
- ✓ Casos especiais (aulas online, salas específicas)


---

## 6. Conclusão

### Resultados Alcançados

Este projeto demonstrou com sucesso a aplicação de **Constraint Satisfaction Problems (CSP)** na resolução de um problema real de escalonamento de horários académicos. Os principais resultados incluem:

1. ✓ **Implementação funcional** de um solver CSP usando `python-constraint`
2. ✓ **Modelação completa** do problema com variáveis, domínios e restrições
3. ✓ **Aplicação rigorosa** de hard constraints e soft constraints
4. ✓ **Descoberta de múltiplas soluções** válidas
5. ✓ **Seleção automática** da melhor solução através de função de avaliação
6. ✓ **Visualização clara** dos resultados em formato tabular

### Contribuições Académicas

O projeto ilustra conceitos fundamentais de Inteligência Artificial:

- **Representação de conhecimento**: Modelação de um problema real como CSP
- **Técnicas de pesquisa**: Backtracking com constraint propagation
- **Otimização**: Função de avaliação para seleção de soluções
- **Engenharia de software**: Código estruturado e documentado

### Aprendizagens

**Pontos Fortes da Abordagem CSP:**

- Separação clara entre modelo (restrições) e algoritmo (solver)
- Facilidade de adicionar novas restrições
- Garantia de validade das soluções encontradas
- Bibliotecas maduras e bem documentadas

**Desafios Encontrados:**

- Equilíbrio entre hard e soft constraints
- Tempo de execução cresce com a complexidade
- Necessidade de função de avaliação bem calibrada

### Conclusão Final

A utilização de CSP para escalonamento de horários demonstrou ser uma abordagem **eficaz, elegante e extensível**. A biblioteca `python-constraint` revelou-se uma ferramenta poderosa para prototipagem rápida e ensino de conceitos de IA.

O projeto alcançou os seus objetivos de forma satisfatória, produzindo horários válidos que respeitam todas as restrições obrigatórias e otimizam critérios de qualidade. A metodologia desenvolvida pode ser facilmente adaptada a outros problemas de escalonamento ou alocação de recursos.

---

**Fim do Projeto**

**Mestrado em Inteligência Artificial Aplicada - IPCA - 2025/2026**